#### Audio Classification Data Preprocessing

#### Observation
Here Librosa converts the signal to mono, meaning the channel will alays be 1

### Extract Features
Here we will be using Mel-Frequency Cepstral Coefficients(MFCC) from the audio 
samples.
The MFCC summarises the frequency distribution across the window size, so it is possible to analyse both the frequency and time characteristics of the sound. These audio representations will allow us to identify features for classification.

In [1]:
#### Extracting MFCC's For every audio file
import pandas as pd
import os
import librosa
import numpy as np
from sklearn.preprocessing import normalize

# breathing_dataset_path=r'C:\Users\ujjwol\Desktop\Masters\Second Year-EURECOM courseware\Semester Project\Second_DiCOVA_Challenge\DICOVA\Second_DiCOVA_Challenge_Dev_Data_Release\AUDIO\breathing'
breathing_dataset_path=r'C:\Users\ujjwol\Desktop\data\breathing'
test_breathing_dataset_path=r'C:\Users\ujjwol\Desktop\Masters\Second Year-EURECOM courseware\Semester Project\Second_DiCOVA_Challenge\Second_DiCOVA_Challenge_Test_Data_Release\AUDIO\breathing'

cough_dataset_path='AUDIO/cough'
test_cough_dataset_path='TEST_AUDIO/cough'


speech_dataset_path='AUDIO/breathing'
test_speech_dataset_path='TEST_AUDIO/breathing'


In [2]:
# metadata=pd.read_csv(r'C:\Users\ujjwol\Downloads\metadata.csv')
metadata=pd.read_csv(r'C:\Users\ujjwol\Desktop\data\augmented_all_metadata.csv')
metadata.head()

,SUB_ID,COVID_STATUS,GENDER
0,NPHYFCYK,n,m
1,NPHYFCYK_noise_injected,n,m
2,NPHYFCYK_pitch_shifted,n,m
3,NPHYFCYK_speed_changed,n,m
4,HTCFAFDO,n,m


In [3]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """
    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [4]:
# def features_extractor(file_name):
#     audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
#     mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
#     mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
#     return mfccs_scaled_features
    

In [5]:
def features_extractor(file_name):
    y_cut, sr = librosa.load(file_name, res_type='kaiser_fast')
    sr=28000
    max_size=1500 #my max audio file feature width
    stft = padding(np.abs(librosa.stft(y_cut, n_fft=255, hop_length= 512)), 128, max_size)
    MFCCs = padding(librosa.feature.mfcc(y_cut, n_fft=255, hop_length=512,n_mfcc=128),128,max_size)
    spec_centroid = librosa.feature.spectral_centroid(y=y_cut, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y_cut, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y_cut, sr=sr)
    #Now the padding part
    image = np.array([padding(normalize(spec_bw),1, max_size)]).reshape(1,max_size)
    image = np.append(image,padding(normalize(spec_centroid),1, max_size), axis=0) 
#repeat the padded spec_bw,spec_centroid and chroma stft until they are stft and MFCC-sized
    for i in range(0,9):
        image = np.append(image,padding(normalize(spec_bw),1, max_size), axis=0)
        image = np.append(image, padding(normalize(spec_centroid),1, max_size), axis=0)
        image = np.append(image, padding(normalize(chroma_stft),12, max_size), axis=0)
    image=np.dstack((image,np.abs(stft)))
    image=np.dstack((image,MFCCs))
    return image

In [6]:
#example_feature = features_extractor('AUGMENTED_AUDIO/breathing/ABEEJBTZ.flac')

In [7]:
#print(example_feature)

In [8]:

from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(breathing_dataset_path),str(row["SUB_ID"]+'.flac'))
    final_class_labels=row["COVID_STATUS"]
    if os.path.isfile(file_name):
        data=features_extractor(file_name)
        extracted_features.append([data,final_class_labels])
    else:
        print("ERROR to find the file",file_name)

0it [00:00, ?it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
1it [00:01,  1.09s/it]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
2it [00:01,  1.65it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel fre

25it [00:07,  2.78it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
26it [00:08,  2.65it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
27it [00:08,  2.37it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in 

49it [00:19,  2.52it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
50it [00:19,  2.90it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
51it [00:19,  3.24it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in 

74it [00:25,  5.24it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
75it [00:25,  5.60it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
76it [00:26,  6.41it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in 

99it [00:31,  3.60it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
100it [00:31,  4.06it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
101it [00:32,  3.08it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected i

...

3741it [17:28,  4.78it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
3742it [17:28,  4.43it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
3743it [17:28,  4.24it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detect

3765it [17:35,  3.35it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
3766it [17:35,  3.71it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
3767it [17:35,  4.13it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detect

3789it [17:40,  7.24it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
3792it [17:40, 11.17i

3814it [17:46,  4.22it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
3815it [17:46,  4.19it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis

3838it [17:52,  3.61it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
3839it [17:52,  3.68it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
3840it [17:52,  4.26it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detect

In [16]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()


In [17]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [18]:
#X.shape

In [19]:
#y

In [20]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [21]:
#y

In [25]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

MemoryError: Unable to allocate 1020. MiB for an array with shape (232, 128, 1500, 3) and data type float64

In [15]:
#X_train

In [16]:
#y

In [17]:
#X_train.shape

In [18]:
#X_test.shape

In [19]:
#y_train.shape

In [20]:
#y_test.shape

### Model Creation

In [21]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [23]:
### No of classes
num_labels=y.shape[1]

In [24]:
num_labels

2

In [25]:
# #FIRST MODEL FROM VIDEO
# model=Sequential()
# ###first layer
# model.add(Dense(100,input_shape=(40,)))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# ###second layer
# model.add(Dense(200))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# ###third layer
# model.add(Dense(100))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

# ###final layer
# model.add(Dense(num_labels))
# model.add(Activation('softmax'))

In [26]:
from tensorflow.keras import models, layers
# FROM medium tutorial
input_shape=(128,1500,3)
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [83]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 126, 1498, 32)     896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 63, 749, 32)      0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 63, 749, 32)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 61, 747, 64)       18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 30, 373, 64)      0         
 2D)                                                             
                                                                 
 dropout_10 (Dropout)        (None, 30, 373, 64)      

In [27]:
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [28]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 40
num_batch_size = 50

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/40
16/16 [==============================] - ETA: 0s - loss: 5.9785 - accuracy: 0.7319 
Epoch 00001: val_loss improved from inf to 0.66765, saving model to saved_models\audio_classification.hdf5
16/16 [==============================] - 289s 17s/step - loss: 5.9785 - accuracy: 0.7319 - val_loss: 0.6677 - val_accuracy: 0.8290
Epoch 2/40
16/16 [==============================] - ETA: 0s - loss: 0.5506 - accuracy: 0.8212 
Epoch 00002: val_loss improved from 0.66765 to 0.48842, saving model to saved_models\audio_classification.hdf5
16/16 [==============================] - 264s 16s/step - loss: 0.5506 - accuracy: 0.8212 - val_loss: 0.4884 - val_accuracy: 0.8290
Epoch 3/40
16/16 [==============================] - ETA: 0s - loss: 0.4403 - accuracy: 0.8199 
Epoch 00003: val_loss improved from 0.48842 to 0.47456, saving model to saved_models\audio_classification.hdf5
16/16 [==============================] - 264s 16s/step - loss: 0.4403 - accuracy: 0.8199 - val_loss: 0.4746 - val_accuracy: 

Epoch 30/40
16/16 [==============================] - ETA: 0s - loss: 0.0095 - accuracy: 0.9987 
Epoch 00030: val_loss did not improve from 0.46668
16/16 [==============================] - 197s 12s/step - loss: 0.0095 - accuracy: 0.9987 - val_loss: 2.3872 - val_accuracy: 0.7720
Epoch 31/40
16/16 [==============================] - ETA: 0s - loss: 0.0142 - accuracy: 0.9961 
Epoch 00031: val_loss did not improve from 0.46668
16/16 [==============================] - 196s 12s/step - loss: 0.0142 - accuracy: 0.9961 - val_loss: 2.2509 - val_accuracy: 0.7720
Epoch 32/40
16/16 [==============================] - ETA: 0s - loss: 0.0066 - accuracy: 0.9987 
Epoch 00032: val_loss did not improve from 0.46668
16/16 [==============================] - 222s 14s/step - loss: 0.0066 - accuracy: 0.9987 - val_loss: 2.4800 - val_accuracy: 0.7927
Epoch 33/40
16/16 [==============================] - ETA: 0s - loss: 0.0364 - accuracy: 0.9858 
Epoch 00033: val_loss did not improve from 0.46668
16/16 [============

In [29]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.772020697593689


In [30]:
#model.predict(X_test)

array([[1.00000000e+00, 5.24208132e-31],
       [1.00000000e+00, 7.77053287e-15],
       [1.00000000e+00, 4.08812900e-10],
       [1.00000000e+00, 8.96895891e-11],
       [1.00000000e+00, 4.95241403e-10],
       [1.00000000e+00, 0.00000000e+00],
       [9.99989986e-01, 1.00515526e-05],
       [1.00000000e+00, 8.69554276e-24],
       [1.00000000e+00, 8.69729132e-26],
       [1.00000000e+00, 1.44926737e-10],
       [9.46835458e-01, 5.31646013e-02],
       [8.87861907e-01, 1.12138145e-01],
       [1.00000000e+00, 1.92742211e-11],
       [1.00000000e+00, 1.63855388e-12],
       [1.00000000e+00, 7.36394480e-18],
       [1.00000000e+00, 4.26390923e-10],
       [9.95353460e-01, 4.64660302e-03],
       [7.07556546e-06, 9.99992967e-01],
       [4.17742848e-01, 5.82257092e-01],
       [1.00000000e+00, 2.09295086e-15],
       [1.00000000e+00, 1.69755337e-19],
       [1.00000000e+00, 3.69224096e-09],
       [1.00000000e+00, 5.23630195e-10],
       [1.00000000e+00, 2.10935436e-11],
       [1.000000

### Testing Some Test Audio Data

Steps
- Preprocess the new audio data
- predict the classes
- Invere transform your Predicted Label

In [ ]:
# filename="AUGMENTED_AUDIO/breathing/GEGERRDK.flac"
# audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
# mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
# mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

# print(mfccs_scaled_features)
# mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
# predicted=model.predict(mfccs_scaled_features)
# predicted_label=np.argmax(predicted,axis=1)
# print(predicted_label)
# prediction_class = labelencoder.inverse_transform(predicted_label) 
# print(prediction_class)

In [ ]:
#STARTING EVALUATION ON BLIND TEST DATA

In [31]:
test_metadata_breathing=pd.read_csv(r'C:\Users\ujjwol\Downloads\test_metadata_breathing.csv')
test_metadata_breathing.head()

,SUB_ID,COVID_STATUS
0,EKANDFRP,n
1,QXIDXPCN,n
2,RJNHLHRD,n
3,ACACIEDW,n
4,JVPUNCRM,n


In [32]:

extracted_features=[]
for index_num,row in tqdm(test_metadata_breathing.iterrows()):
    file_name = os.path.join(os.path.abspath(test_breathing_dataset_path),str(row["SUB_ID"]+'.flac'))
    final_class_labels=row["COVID_STATUS"]
    if os.path.isfile(file_name):
        data=features_extractor(file_name)
        extracted_features.append([data,final_class_labels])
    else:
        print("ERROR to find the file", file_name)
        

0it [00:00, ?it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
1it [00:00,  1.21it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
2it [00:01,  1.75it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel fre

48it [00:29,  1.51it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
49it [00:30,  1.41it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
50it [00:30,  1.51it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in 

97it [01:01,  1.51it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
98it [01:02,  1.32it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
99it [01:02,  1.62it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in 

121it [01:16,  1.44it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
122it [01:16,  1.54it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
123it [01:17,  1.59it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected 

145it [01:30,  1.89it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
146it [01:30,  1.93it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
147it [01:31,  1.89it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected 

...

337it [03:25,  1.53it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
338it [03:26,  1.65it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
339it [03:26,  1.65it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected 

385it [03:55,  1.76it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
386it [03:56,  1.51it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
387it [03:57,  1.41it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected 

409it [04:10,  1.86it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
410it [04:10,  1.89it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
411it [04:11,  1.83it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected 

433it [04:24,  1.66it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
434it [04:24,  1.74it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
435it [04:25,  1.56it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected 

457it [04:37,  1.55it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
458it [04:38,  1.49it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "
459it [04:38,  1.58it/s]c:\users\ujjwol\appdata\local\programs\python\python37\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected 

In [33]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
### Label Encoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))


In [34]:
test_accuracy=model.evaluate(X,y)
print(test_accuracy[1])

15/15 [==============================] - 38s 2s/step - loss: 2.4317 - accuracy: 0.8132
0.8131634593009949


In [ ]:
print(test_accuracy)
